In [1]:
!python --version

Python 3.9.16


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

train_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/train.csv'
test_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/test.csv'

Mounted at /content/gdrive


In [3]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
snowball_stemmer_obj = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

In [4]:
import pandas as pd
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)

In [5]:
df_train

,class,title,text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [6]:
# TODO - make two dimensianal container for tokens [sentence_index, token_index]
# TODO - regex

In [69]:
import re

    # TODO money_in_bucks_regex - apply last examples to it
money_in_bucks_regex = r'\$\d+(?:\,\d{3})?(?:\.\d+)?\s?(?:trillion|billion|million|thousand|trln|bn|mn|th)?' # $15trln ; $15 trln ; $15,000 ; $15 ; $15.2 ; $1.8bn ; $1 mn ; $0.49 ; ?63.5 million dollars? ... ; $1.033 billion
metric_weight_regex = r'\d+(?:kg|mg|g|t)' # 81kg
# телефоны - The phone number for the New York sports desk is (212) 621-1630 ; (800-838-4616) ; 
# дата - Aug. 17 ; Aug 17 ; Sept. 11 ; Jan. 2000 ; стандартные даты
# сокращения через точку - M.V.P. ; S.E.C.
# время - 1:54.04 ; 1:30 pm ; 9:03
# веб-адреса - http://yourap.org/ ; For-Side.com
# компании с сокращениями - Google inc. ; General Motors Corp. ; Halliburton Co. ; Louisiana-Pacific Corp.
# пунктуация(тоже выносим в токены) - " - " __ ": " __ "; " __ "! " __ "? " __ ". " __ ", "
# email - bebra-shmebrikovna.reks3-4-4@yarus34.ru
not_space_regex = r'[^\s]+' # any not space symbol sequence

          # убрать мусор в виде \\ и прочих не несущих смысла знаков

# распределить по приоритету, вроде бы та регулярка что первее, матчится в первую очередь
regex_to_take_tokens = money_in_bucks_regex + '|' + metric_weight_regex + '|' + not_space_regex

# Проверка регекса баксов

In [37]:
test_example_bucks = df_train['title'][1538] + '. ' + df_train['text'][1538]

In [38]:
test_example_bucks

'European/US private equity consortium buys Intelsat in deal valued at \\$5bn. An unusual alliance of US and European private equity firms has won the auction to buy Bermuda-based satellite business Intelsat. Europeans Apax and Permira and US firms Apollo Management and Madison Dearborn will pay \\$3bn for the business and assume \\$2bn ...'

In [44]:
test_result_bucks = re.findAll(money_in_bucks_regex, test_example_bucks)

In [45]:
test_result_bucks

['European/US private equity consortium buys Intelsat in deal valued at \\',
 '. An unusual alliance of US and European private equity firms has won the auction to buy Bermuda-based satellite business Intelsat. Europeans Apax and Permira and US firms Apollo Management and Madison Dearborn will pay \\',
 ' for the business and assume \\',
 ' ...']

# Проверка регекса веса

In [ ]:
test_example_weight = df_train['title'][1599] + '. ' + df_train['text'][1599]

In [48]:
test_example_weight

'Iliadis Takes Greece #39;s Second Gold with Judo Win. ATHENS (Reuters) - Greek teenager Ilias Iliadis swept up the Olympic host country #39;s second gold medal on Tuesday, winning the men #39;s judo under 81kg category. '

In [46]:
test_result_weight = re.findAll(metric_weight_regex, test_example_weight)

In [47]:
test_result_weight

['Iliadis Takes Greece #39;s Second Gold with Judo Win. ATHENS (Reuters) - Greek teenager Ilias Iliadis swept up the Olympic host country #39;s second gold medal on Tuesday, winning the men #39;s judo under ',
 ' category. ']

# Проверка итогового регекса

In [61]:
regex_to_take_tokens

'\\$\\d+(?:\\,\\d{3})?(?:\\.\\d+)?\\s?(?:trln|bn|mn|th|trillion|billion|million|thousand)?|\\d+(?:kg|mg|g|t)|[^\\s]+'

In [66]:
test_example_bucks

'European/US private equity consortium buys Intelsat in deal valued at \\$5bn. An unusual alliance of US and European private equity firms has won the auction to buy Bermuda-based satellite business Intelsat. Europeans Apax and Permira and US firms Apollo Management and Madison Dearborn will pay \\$3bn for the business and assume \\$2bn ...'

In [70]:
final_result = re.findall(regex_to_take_tokens, test_example_bucks + 'valued at \\$5 bn. $5 billion $5 billion $5 thousand An unusual alliance') 
# чёт не то, доделать (почему $5 bn в два токена бьётся)

In [71]:
final_result

['European/US',
 'private',
 'equity',
 'consortium',
 'buys',
 'Intelsat',
 'in',
 'deal',
 'valued',
 'at',
 '\\$5bn.',
 'An',
 'unusual',
 'alliance',
 'of',
 'US',
 'and',
 'European',
 'private',
 'equity',
 'firms',
 'has',
 'won',
 'the',
 'auction',
 'to',
 'buy',
 'Bermuda-based',
 'satellite',
 'business',
 'Intelsat.',
 'Europeans',
 'Apax',
 'and',
 'Permira',
 'and',
 'US',
 'firms',
 'Apollo',
 'Management',
 'and',
 'Madison',
 'Dearborn',
 'will',
 'pay',
 '\\$3bn',
 'for',
 'the',
 'business',
 'and',
 'assume',
 '\\$2bn',
 '...valued',
 'at',
 '\\$5',
 'bn.',
 '$5 billion',
 '$5 billion',
 '$5 thousand',
 'An',
 'unusual',
 'alliance']

# Песочница

In [52]:
dummy_word_regex = r'[\s]+'
dummy_split = re.split(dummy_word_regex, test_example_weight)

In [53]:
dummy_split

['Iliadis',
 'Takes',
 'Greece',
 '#39;s',
 'Second',
 'Gold',
 'with',
 'Judo',
 'Win.',
 'ATHENS',
 '(Reuters)',
 '-',
 'Greek',
 'teenager',
 'Ilias',
 'Iliadis',
 'swept',
 'up',
 'the',
 'Olympic',
 'host',
 'country',
 '#39;s',
 'second',
 'gold',
 'medal',
 'on',
 'Tuesday,',
 'winning',
 'the',
 'men',
 '#39;s',
 'judo',
 'under',
 '81kg',
 'category.',
 '']